In [1]:
from ipynb.fs.full.metrics import exos_running_time_df
from ipynb.fs.full.metrics import compute_accuracy

In [2]:
n_streams = 2
gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/two'
gt_filename = '2_100K_Case1.pkl'
result_folder = '/home/epanjei/Codes/EXOS/sources/experiments/pickles/nstreams'
matched, matched2, n_outliers = compute_accuracy(gt_folder= gt_folder, 
                                                 gt_filename= gt_filename, 
                                                 result_folder=result_folder, 
                                                 result_filename=gt_filename,
                                                 n_streams=n_streams, 
                                                 window_size=1000,
                                                 score_precision = 0.1)
print(len(matched)/n_outliers * 100)
print(len(matched2)/n_outliers * 100)

63.14999999999999
61.199999999999996


In [3]:
n_streams = 4
gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/four'
gt_filename = '4_100K_Case1.pkl'
result_folder = '/home/epanjei/Codes/EXOS/sources/experiments/pickles/nstreams'
matched, matched2, n_outliers = compute_accuracy(gt_folder= gt_folder, 
                                                 gt_filename= gt_filename, 
                                                 result_folder=result_folder, 
                                                 result_filename=gt_filename,
                                                 n_streams=n_streams, 
                                                 window_size=1000,
                                                 score_precision = 0.1)
print(len(matched)/n_outliers * 100)
print(len(matched2)/n_outliers * 100)

73.0
62.075


In [4]:
n_streams = 8
gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/eight'
gt_filename = '8_100K_Case1.pkl'
result_folder = '/home/epanjei/Codes/EXOS/sources/experiments/pickles/nstreams'
matched, matched2, n_outliers = compute_accuracy(gt_folder= gt_folder, 
                                                 gt_filename= gt_filename, 
                                                 result_folder=result_folder, 
                                                 result_filename=gt_filename,
                                                 n_streams=n_streams, 
                                                 window_size=1000,
                                                 score_precision = 0.1)
print(len(matched)/n_outliers * 100)
print(len(matched2)/n_outliers * 100)

74.775
61.275


In [5]:
n_streams = 16
gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/sixteen'
gt_filename = '16_100K_Case1.pkl'
result_folder = '/home/epanjei/Codes/EXOS/sources/experiments/pickles/nstreams'
matched, matched2, n_outliers = compute_accuracy(gt_folder= gt_folder, 
                                                 gt_filename= gt_filename, 
                                                 result_folder=result_folder, 
                                                 result_filename=gt_filename,
                                                 n_streams=n_streams, 
                                                 window_size=1000,
                                                 score_precision = 0.1)
print(len(matched)/n_outliers * 100)
print(len(matched2)/n_outliers * 100)

80.13125
62.08125


In [6]:
n_streams = 24
gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/twentyfour'
gt_filename = '24_100K_Case1.pkl'
result_folder = '/home/epanjei/Codes/EXOS/sources/experiments/pickles/nstreams'
matched, matched2, n_outliers = compute_accuracy(gt_folder= gt_folder, 
                                                 gt_filename= gt_filename, 
                                                 result_folder=result_folder, 
                                                 result_filename=gt_filename,
                                                 n_streams=n_streams, 
                                                 window_size=1000,
                                                 score_precision = 0.1)
print(len(matched)/n_outliers * 100)
print(len(matched2)/n_outliers * 100)

81.9375
62.20833333333333


In [7]:
n_streams = 32
gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/thirtytwo'
gt_filename = '32_100K_Case1.pkl'
result_folder = '/home/epanjei/Codes/EXOS/sources/experiments/pickles/nstreams'
matched, matched2, n_outliers = compute_accuracy(gt_folder= gt_folder, 
                                                 gt_filename= gt_filename, 
                                                 result_folder=result_folder, 
                                                 result_filename=gt_filename,
                                                 n_streams=n_streams, 
                                                 window_size=1000,
                                                 score_precision = 0.1)
print(len(matched)/n_outliers * 100)
print(len(matched2)/n_outliers * 100)

83.240625
60.9625
